In [744]:
import pandas as pd
import numpy as np
import re

import sys

sys.path.append("../src/biblical_scripts/pipelines/data_science")
sys.path.append("/Users/kipnisal/DS/BiblicalScripts/bib-scripts/src")

from AuthorshipAttribution.AuthAttLib import *

from tqdm import tqdm

import logging
logging.basicConfig(level=logging.INFO)

In [745]:
project_dir = "/Users/kipnisal/DS/BiblicalScripts/bib-scripts"

from kedro.framework.context import load_context
cnx = load_context(project_dir)

from kedro.config import ConfigLoader

conf_paths = [path +'/'+ "conf/base",path +'/'+ "conf/local"]
conf_loader = ConfigLoader(conf_paths)
conf_catalog = conf_loader.get("catalog*", "catalog*/**")

conf_paths = [path + '/' + "conf/base",path + '/' + "conf/local"]
conf_loader = ConfigLoader(conf_paths)
parameters = conf_loader.get("parameters*", "parameters*/**")


/usr/local/lib/python3.8/site-packages/kedro/framework/context/context.py:663: DeprecationWarning: `kedro.framework.context.load_context` is now deprecated in favour of `KedroSession.load_context` and will be removed in Kedro 0.18.0.
INFO:anyconfig:Loading: /Users/kipnisal/DS/BiblicalScripts/bib-scripts/pyproject.toml
INFO:anyconfig:Loading: /Users/kipnisal/DS/BiblicalScripts/bib-scripts/conf/base/catalog.yml
INFO:anyconfig:Loading: /Users/kipnisal/DS/BiblicalScripts/bib-scripts/conf/base/parameters.yml


# Text Processing

In [747]:
data = pd.read_csv(path + '/' + 'data/02_intermediate/oshd_parsed.csv')
from biblical_scripts.pipelines.data_engineering.TextProcessing import TextProcessing
tp = TextProcessing(extract_prefix=True, ng_range=(2,2))
data_proc = tp.proc_ng(data)

INFO:root:Extracting prefixes and suffixes
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

# Conversion

In [936]:
from biblical_scripts.extras.Convert import Convert
convert = Convert(data)

In [937]:
convert.to_term(('120', '1456'))

['אָדָ֣ם', 'גֵּהָ֑ה']

In [939]:
vocab = pd.read_csv("/Users/kipnisal/DS/BiblicalScripts/bib-scripts/data/05_model_input/vocabulary.csv" )

In [1025]:
from sklearn.model_selection import KFold

In [961]:
vocab.loc[1,'feature']

"('c', '<Np>')"

# BS

In [1030]:
measures = ['HC', 'Fisher', 'chisq']
r"|".join(measures)

'HC|Fisher|chisq'

# Models

In [855]:
data_proc = pd.read_csv(path + '/' + 'data/04_feature/data_proc.csv')

In [880]:
import pandas as pd
import numpy as np
import logging
%reload_ext autoreload
from biblical_scripts.pipelines.data_science.AuthorshipAttribution.AuthAttLib import AuthorshipAttributionDTM
from biblical_scripts.pipelines.data_science.AuthorshipAttribution.MultiDoc import CompareDocs
from biblical_scripts.pipelines.data_science.nodes import (build_vocab, compute_sim)
#%load_ext autoreload
#%autoreload 2

data_raw = pd.read_csv(path + '/' + 'data/02_intermediate/oshd_parsed.csv')
data_proc = pd.read_csv(path + '/' + 'data/04_feature/data_proc.csv')
#vocab = pd.read_csv(path + '/' + 'data/05_model_input/vocabulary.csv')
vocab = build_vocab(data_proc, parameters['vocab'], parameters['known_authors'])

INFO:root:Constructed vocabulary with 960 features


In [878]:
def compute_sim(data, vocabulary, params, known_authors) :
    ds = data.rename(columns = {'feature' : 'term', 'chapter' : 'doc_id'}).dropna()
    model = AuthorshipAttributionDTM(ds, params, vocab=listb(vocabulary.feature.values))
    return model.compute_inter_similarity(LOO = True, wrt_authors=known_authors)

In [882]:
params = {}; [params.update(p) for p in parameters['model']]

md = CompareDocs(vocabulary=vocab.feature.tolist())
#ds=data_proc[['author', 'feature', 'token_id']].groupby(['author', 'feature']).count().reset_index()
ds=data_proc[['author', 'feature', 'token_id', 'chapter']]
ds=ds.rename(columns = {'chapter' : 'doc_id'})
ds['doc_id'] += ' by '
ds['doc_id'] += ds['author'] #sometimes there are multiple authors per chapter
    
known_authors = parameters['known_authors']
train_data = {}
for auth in known_authors :
    train_data[auth] = ds[ds.author==auth]

md.fit(train_data)
df_res1 = pd.DataFrame()
for doc_id in ds.doc_id.unique() :
    tested_doc = ds[ds.doc_id==doc_id]
    auth = tested_doc.author.values[0]
    df_rec = md.test_doc(tested_doc, of_cls = auth)
    r = df_rec.iloc[:,df_rec.columns.str.contains(r'HC|Fisher|chisq')].mean()
    r['doc_id'] = doc_id
    r['author'] = auth
    r['len'] = len(tested_doc)
    df_res1 = df_res1.append(r, ignore_index=True)

DEBUG:root:Processing Dtr...
DEBUG:root:Found 3366 terms.
DEBUG:root:Processing DtrH...
DEBUG:root:Found 3938 terms.
DEBUG:root:Processing P...
DEBUG:root:Found 6561 terms.
DEBUG:root:Doc contains 105.0 terms.
DEBUG:root:Doc contains 210.0 terms.
DEBUG:root:Doc contains 118.0 terms.
DEBUG:root:Doc contains 97.0 terms.
DEBUG:root:Doc contains 98.0 terms.
DEBUG:root:Doc contains 118.0 terms.
DEBUG:root:Doc contains 203.0 terms.
DEBUG:root:Doc contains 277.0 terms.
DEBUG:root:Doc contains 107.0 terms.
DEBUG:root:Doc contains 164.0 terms.
DEBUG:root:Doc contains 271.0 terms.
DEBUG:root:Doc contains 183.0 terms.
DEBUG:root:Doc contains 146.0 terms.
DEBUG:root:Doc contains 88.0 terms.
DEBUG:root:Doc contains 106.0 terms.
DEBUG:root:Doc contains 147.0 terms.
DEBUG:root:Doc contains 167.0 terms.
DEBUG:root:Doc contains 242.0 terms.
DEBUG:root:Doc contains 198.0 terms.
DEBUG:root:Doc contains 224.0 terms.
DEBUG:root:Doc contains 20.0 terms.
DEBUG:root:Doc contains 195.0 terms.
DEBUG:root:Doc co

In [932]:
df = df_res1.melt(['author', 'doc_id', 'len'])

def _eval_succ(df) :
    df['wrt_author'] = df['variable'].str.extract(r'([^:]+):')
    idx_min = df.groupby(['doc_id', 'author'])['value'].idxmin()
    res_min = df.loc[idx_min, :].rename(columns={'wrt_author' : 'most_sim'})
    res_min.loc[:, 'succ'] = res_min.author == res_min.most_sim
    return res_min

value = report_params['value']
df1 = df[df['variable'].str.contains(f":{value}")]
df1 = df1[df1.author.isin(known_authors)].reset_index()

res = _eval_succ(df1)
res['acc'] = res.succ.mean()
res['param'] = str(parameters)

In [935]:
ds = pd.read_csv("../data/05_model_input/vocabulary.csv")
ds[~ds.feature.str.contains(r"\[[a-zA-Z0-9]+\]")]

,feature
0,103
1,1086
2,1104
3,1115
4,1129
...,...
636,8484
637,8519
638,8535
639,887


In [876]:
data_raw[data_raw.chapter == '2Kgs.23']

array(['21, 2 Kings 23 :21-30', 'DtrH'], dtype=object)

# Feature Selection

In [763]:
import pandas as pd
import numpy as np
import logging
%reload_ext autoreload
from biblical_scripts.pipelines.data_science.AuthorshipAttribution.AuthAttLib import AuthorshipAttributionDTM
from biblical_scripts.pipelines.data_science.AuthorshipAttribution.MultiDoc import CompareDocs
from biblical_scripts.pipelines.data_science.nodes import (build_vocab, compute_sim)
#%load_ext autoreload
#%autoreload 2

In [764]:
#data:
data_proc = pd.read_csv(path + '/' + 'data/04_feature/data_proc.csv')
vocab = build_vocab(data_proc, parameters['vocab'], parameters['known_authors'])

INFO:root:Constructed a vocabulary of 960 features


In [765]:
# using discriminating features

md = CompareDocs(vocabulary=vocab.feature.to_list())
#ds=data_proc[['author', 'feature', 'token_id']].groupby(['author', 'feature']).count().reset_index()
ds=data_proc[['author', 'feature', 'token_id']]

Dtr = ds[ds.author=='Dtr']
DtrH = ds[ds.author=='DtrH']
P = ds[ds.author=='P']

md.fit({'Dtr' : Dtr, 'DtrH' : DtrH, 'P' : P})
df_disc = md.HCT(stbl=True)
feat = df_disc[df_disc.thresh].index.to_list()
terms = [convert.to_term(f) for f in feat]

DEBUG:root:Processing Dtr...
DEBUG:root:Found 3366 terms.
DEBUG:root:Processing DtrH...
DEBUG:root:Found 3938 terms.
DEBUG:root:Processing P...
DEBUG:root:Found 6561 terms.
INFO:root:Using multinomial tests. May be slow.


In [757]:
ds = data_proc.drop('term', axis=1).rename(columns = {'chapter' : 'doc_id', 'feature' : 'term'}).dropna()
ds_red = ds[ds.term.isin(feat)]
ds_red[ds_red.groupby('doc_id').term.transform(pd.Series.count)>0]
knwon_authors = parameters['known_authors']
model = AuthorshipAttributionDTM(ds_red, **param, vocab=feat)
df_res=model.compute_inter_similarity(LOO = True, wrt_authors=known_authors)

INFO:root:Creating author-model for Chr1...
INFO:root:Found 9 documents and 853 relevant tokens.
INFO:root:Creating author-model for 12, 1 Kings 8-10; 11:41-43...
INFO:root:Found 1 documents and 434 relevant tokens.
INFO:root:Creating author-model for DtrH...
INFO:root:Found 14 documents and 2457 relevant tokens.
INFO:root:Creating author-model for Ark1...
INFO:root:Found 3 documents and 306 relevant tokens.
INFO:root:Creating author-model for Chr2...
INFO:root:Found 8 documents and 705 relevant tokens.
INFO:root:Creating author-model for 21, 2 Kings 23 :21-30...
INFO:root:Found 1 documents and 71 relevant tokens.
INFO:root:Creating author-model for 3, 2 Sam 6:1-11...
INFO:root:Found 1 documents and 44 relevant tokens.
INFO:root:Creating author-model for Ark2...
INFO:root:Found 1 documents and 101 relevant tokens.
INFO:root:Creating author-model for 5, 2 Sam 6:12-23...
INFO:root:Found 1 documents and 57 relevant tokens.
INFO:root:Creating author-model for 6, 2 Sam 7...
INFO:root:Found 

In [769]:
df_rec

,n,T,Dtr:n,Dtr:T,DtrH:n,DtrH:T,P:n,P:T,tested:n,tested:T,...,DtrH:Fisher,DtrH:HC,DtrH:chisq,DtrH:affinity,P:pval,P:score,P:Fisher,P:HC,P:chisq,P:affinity
feature,,,,,,,,,,,,,,,,,,,,,
1,95,8684,32,2048,48,2457,15,4179,0.0,54.0,...,134.049533,0.926417,NaN,0.0,1.000000,-0.000000,185.55043,2.064066,NaN,0.0
1004,123,8684,17,2048,82,2457,24,4179,4.0,54.0,...,134.049533,0.926417,NaN,-0.0,0.000326,16.058764,185.55043,2.064066,NaN,-1.0
1115,8,8684,2,2048,6,2457,0,4179,0.0,54.0,...,134.049533,0.926417,NaN,0.0,1.000000,-0.000000,185.55043,2.064066,NaN,-0.0
1121,364,8684,32,2048,81,2457,251,4179,3.0,54.0,...,134.049533,0.926417,NaN,-0.0,1.000000,-0.000000,185.55043,2.064066,NaN,0.0
1129,33,8684,6,2048,27,2457,0,4179,0.0,54.0,...,134.049533,0.926417,NaN,0.0,1.000000,-0.000000,185.55043,2.064066,NaN,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5207,14,8684,0,2048,0,2457,14,4179,0.0,54.0,...,134.049533,0.926417,NaN,-0.0,1.000000,-0.000000,185.55043,2.064066,NaN,0.0
5216,11,8684,0,2048,0,2457,11,4179,1.0,54.0,...,134.049533,0.926417,NaN,-0.0,0.140693,3.922349,185.55043,2.064066,NaN,-0.0
5561,12,8684,0,2048,0,2457,12,4179,0.0,54.0,...,134.049533,0.926417,NaN,-0.0,1.000000,-0.000000,185.55043,2.064066,NaN,0.0


In [773]:
md = CompareDocs(vocabulary=feat)
#ds=data_proc[['author', 'feature', 'token_id']].groupby(['author', 'feature']).count().reset_index()
ds=data_proc[['author', 'feature', 'token_id', 'chapter']]
ds=ds.rename(columns = {'chapter' : 'doc_id'})

known_authors = parameters['known_authors']
train_data = {}
for auth in known_authors :
    train_data[auth] = ds[ds.author==auth]

md.fit(train_data)
df_res1 = pd.DataFrame()
for doc_id in ds.doc_id.unique() :
    tested_doc = ds[ds.doc_id==doc_id]
    auth = tested_doc.author.values[0]
    df_rec = md.test_doc(tested_doc, of_cls = auth)
    r = df_rec.iloc[:,df_rec.columns.str.contains(r'HC|Fisher|chisq')].mean()
    r['doc_id'] = doc_id
    r['author'] = auth
    r['len'] = len(tested_doc)
    df_res1 = df_res1.append(r, ignore_index=True)

DEBUG:root:Processing Dtr...
DEBUG:root:Found 2048 terms.
DEBUG:root:Processing DtrH...
DEBUG:root:Found 2457 terms.
DEBUG:root:Processing P...
DEBUG:root:Found 4179 terms.
DEBUG:root:Doc contains 49.0 terms.
DEBUG:root:Doc contains 119.0 terms.
DEBUG:root:Doc contains 68.0 terms.
DEBUG:root:Doc contains 65.0 terms.
DEBUG:root:Doc contains 60.0 terms.
DEBUG:root:Doc contains 74.0 terms.
DEBUG:root:Doc contains 109.0 terms.
DEBUG:root:Doc contains 175.0 terms.
DEBUG:root:Doc contains 71.0 terms.
DEBUG:root:Doc contains 92.0 terms.
DEBUG:root:Doc contains 175.0 terms.
DEBUG:root:Doc contains 124.0 terms.
DEBUG:root:Doc contains 93.0 terms.
DEBUG:root:Doc contains 62.0 terms.
DEBUG:root:Doc contains 57.0 terms.
DEBUG:root:Doc contains 84.0 terms.
DEBUG:root:Doc contains 91.0 terms.
DEBUG:root:Doc contains 159.0 terms.
DEBUG:root:Doc contains 115.0 terms.
DEBUG:root:Doc contains 136.0 terms.
DEBUG:root:Doc contains 16.0 terms.
DEBUG:root:Doc contains 132.0 terms.
DEBUG:root:Doc contains 59

ValueError: All values in `observed` must be nonnegative.

In [793]:
df_eval = df_res1.melt(['author', 'doc_id', 'len'])
df_eval.head(3)

,author,doc_id,len,variable,value
0,"1, 1 Chron 10",1Chr.10,315.0,Dtr:Fisher,140.932326
1,"2, 1 Chron 11",1Chr.11,761.0,Dtr:Fisher,266.941973
2,Chr1,1Chr.12,387.0,Dtr:Fisher,227.374529


In [792]:
df1 = df[df['variable'].str.contains(f":{value}")]
df1.head(3)

,author,doc_id,len,variable,value,wrt_author
63,"1, 1 Chron 10",1Chr.10,315.0,Dtr:HC,1.161644,Dtr
64,"2, 1 Chron 11",1Chr.11,761.0,Dtr:HC,1.367914,Dtr
65,Chr1,1Chr.12,387.0,Dtr:HC,1.858160,Dtr


In [832]:
known_authors = parameters['known_authors']
report_params = parameters['report']
value = report_params['value']
df = df_eval

#def evaluate_accuracy(df : pd.DataFrame, known_authors : List, report_params, parameters) -> float :
def _eval_acc(df) :
    df['wrt_author'] = df['variable'].str.extract(r'([^:]+):')
    idx_min = res1.groupby(['doc_id', 'author'])['value'].idxmin()
    res_min = res1.loc[idx_min, :]
    res_min.loc[:, 'succ'] = res_min.author == res_min.wrt_author
    succ_per_doc = res_min.groupby('doc_id').succ.mean()
    return succ_per_doc.mean()

value = report_params['value']
df1 = df[df['variable'].str.contains(f":{value}")]
df1 = df1[df1.author.isin(known_authors)].reset_index()

acc = _eval_acc(df1)

#rep['param'] = parameters
return pd.DataFrame({'acc' : acc}, index=[parameters])


                                                         acc
{'preprocessing': {'extract_prefix': True, 'ext...  0.638554


In [830]:
df1

,index,author,doc_id,len,variable,value,wrt_author
0,124,DtrH,2Kgs.17,520.0,Dtr:HC,2.220364,Dtr
1,125,DtrH,2Kgs.22,568.0,Dtr:HC,3.010700,Dtr
2,313,DtrH,2Kgs.17,520.0,DtrH:HC,0.643856,DtrH
3,314,DtrH,2Kgs.22,568.0,DtrH:HC,1.272487,DtrH
4,502,DtrH,2Kgs.17,520.0,P:HC,3.731762,P
5,503,DtrH,2Kgs.22,568.0,P:HC,3.273535,P


TypeError: reduction operation 'argmin' not allowed for this dtype

In [741]:
df

,author,doc_id,len,variable,value,wrt_author
0,"1, 1 Chron 10",1Chr.10,315.0,HC (Dtr),1.161644,Dtr
1,"2, 1 Chron 11",1Chr.11,761.0,HC (Dtr),1.367914,Dtr
2,Chr1,1Chr.12,387.0,HC (Dtr),1.858160,Dtr
3,"3, 1 Chron 13",1Chr.13,314.0,HC (Dtr),-0.623731,Dtr
4,"4, 1 Chron 14",1Chr.14,309.0,HC (Dtr),1.358312,Dtr
...,...,...,...,...,...,...
577,Prov,Prov.27,316.0,HC (P),2.561318,P
578,Prov,Prov.28,304.0,HC (P),2.202612,P
579,Prov,Prov.29,269.0,HC (P),2.896388,P
580,Prov,Prov.30,412.0,HC (P),3.781481,P


In [687]:
def eval_accuracy(df, known_authors) :
    res1 = df[df.author.isin(known_authors) & df.wrt_author.isin(known_authors)].reset_index()
    idx_min = res1.groupby(['doc_id', 'author'])['value'].idxmin()
    res_min = res1.loc[idx_min, :]
    res_min.loc[:, 'succ'] = res_min.author == res_min.wrt_author

    succ_per_doc = res_min.groupby('doc_id').succ.mean()
    return succ_per_doc.mean()

df_eval = df_res1.melt(['author', 'doc_id'])
df_eval['wrt_author'] = df_eval['variable'].str.extract(r'HC \((.+)\)')
eval_accuracy(df_eval, known_authors)

0.6385542168674698

In [691]:
eval_accuracy(df_res.rename(columns={'HC' : 'value'}), known_authors)

0.5471698113207547

In [680]:
df_res['len'] = 1000
evaluate_accuracy(df_res, known_authors, 'HC', "")

TypeError: Invalid comparison between dtype=int64 and str

In [677]:
def evaluate_accuracy(df_res: pd.DataFrame, known_authors, min_length_to_report : int, params) -> pd.DataFrame :
    """
    Evaluate accuracy of attribution in min discrepancy manner
    
    Args:
    min_length_to_report    only try to attribute docs of length larger than that
    known_authors           only compare against corpora from this list
    
    Returns:
    average accuracy for each similarity measure
    """
    def prob_of_succ(df, value, known_authors, min_length=min_length_to_report, plot=False) :
        df = df[df.len >= min_length]
        df.loc[:,value] = df[value].astype(float)
        res1 = df[df.author.isin(known_authors) & df.wrt_author.isin(known_authors)].reset_index()
        idx_min = res1.groupby(['doc_id', 'author'])[value].idxmin()
        res_min = res1.loc[idx_min, :]
        res_min.loc[:, 'succ'] = res_min.author == res_min.wrt_author

        succ_per_doc = res_min.groupby('doc_id').succ.mean()
        return succ_per_doc.mean()

    res = {}
    for v in ['HC', 'HC_rank', 'chisq', 'chisq_rank', 'log-likelihood','log-likelihood_rank', 'Fisher']  :
        res[v] = prob_of_succ(df_res, value = v, known_authors=known_authors)
        
    res['params'] = str(params)
    return pd.DataFrame(res, index=[0])
